<a href="https://www.kaggle.com/code/mohamedabobakr0/instacart-market?scriptVersionId=193836040" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [13]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import zipfile
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/instacart-market-basket-analysis/departments.csv.zip
/kaggle/input/instacart-market-basket-analysis/sample_submission.csv.zip
/kaggle/input/instacart-market-basket-analysis/order_products__train.csv.zip
/kaggle/input/instacart-market-basket-analysis/order_products__prior.csv.zip
/kaggle/input/instacart-market-basket-analysis/orders.csv.zip
/kaggle/input/instacart-market-basket-analysis/products.csv.zip
/kaggle/input/instacart-market-basket-analysis/aisles.csv.zip


## 1. Data Exploration and Preprocessing
### a. Load the Data:
 - Load the CSV files into pandas DataFrames.
#### Dateframs

    - `departments`
    - `sample_submission`
    - `order_products__train`
    - `order_products__prior` 
    - `orders`
    - `products`
    - `aisles`
 - Check for missing values and data types.

 ### b. Data Cleaning:
   - Handle missing values.
   - Convert data types (e.g., datetime conversion).
   - Merge DataFrames as needed (e.g., joining order_products with products).

In [14]:
path = "/kaggle/input/instacart-market-basket-analysis/"
files = [
    "departments.csv.zip",
    "order_products__train.csv.zip",
    "order_products__prior.csv.zip",
    "orders.csv.zip",
    "products.csv.zip",
    "aisles.csv.zip"
]
for file in files:
    with zipfile.ZipFile(path+file,"r") as z:
        z.extractall(".")

def read_file(file):
    return pd.read_csv(file)

In [15]:
departments,order_products__train,order_products__prior,orders,products,aisles = \
[read_file(file.split(".zip")[0]) for file in files]

In [16]:
order_products = pd.concat([order_products__train,order_products__prior],ignore_index=True)
orders.drop("eval_set",axis=1,inplace= True)

## New table


- `orders` `drop()` eval_set 
- `order_products` `concat()` order_products__train, order_products__prior
- `departments`
- `products`
- `aisles`

In [17]:
def check_not_null(dataframe):
    return dataframe.notnull().all(axis=1)

def check_data_types(dataframe):
    dataframe.info(show_counts=True)

In [18]:
all_dataframe = {"orders":orders,
                 "order_products":order_products,
                 "departments":departments,
                 "products":products,
                 "aisles":aisles}

In [19]:
for key,df in all_dataframe.items():
    print(f"{key.capitalize()}\n{check_not_null(df).value_counts()}\n")

Orders
True     3214874
False     206209
Name: count, dtype: int64

Order_products
True    33819106
Name: count, dtype: int64

Departments
True    21
Name: count, dtype: int64

Products
True    49688
Name: count, dtype: int64

Aisles
True    134
Name: count, dtype: int64



In [20]:
for key,df in all_dataframe.items():
    print(f"{key.capitalize()}")
    check_data_types(df)
    print("\n")

Orders
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3421083 entries, 0 to 3421082
Data columns (total 6 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   order_id                3421083 non-null  int64  
 1   user_id                 3421083 non-null  int64  
 2   order_number            3421083 non-null  int64  
 3   order_dow               3421083 non-null  int64  
 4   order_hour_of_day       3421083 non-null  int64  
 5   days_since_prior_order  3214874 non-null  float64
dtypes: float64(1), int64(5)
memory usage: 156.6 MB


Order_products
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33819106 entries, 0 to 33819105
Data columns (total 4 columns):
 #   Column             Non-Null Count     Dtype
---  ------             --------------     -----
 0   order_id           33819106 non-null  int64
 1   product_id         33819106 non-null  int64
 2   add_to_cart_order  33819106 non-null  int64
 3   reordere

In [21]:
orders.loc[orders["days_since_prior_order"].isna(),["days_since_prior_order"]] = 0

In [22]:
orders["days_since_prior_order"] = orders["days_since_prior_order"].astype(int)
orders.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3421083 entries, 0 to 3421082
Data columns (total 6 columns):
 #   Column                  Non-Null Count    Dtype
---  ------                  --------------    -----
 0   order_id                3421083 non-null  int64
 1   user_id                 3421083 non-null  int64
 2   order_number            3421083 non-null  int64
 3   order_dow               3421083 non-null  int64
 4   order_hour_of_day       3421083 non-null  int64
 5   days_since_prior_order  3421083 non-null  int64
dtypes: int64(6)
memory usage: 156.6 MB


In [23]:
products = products.merge(aisles).merge(departments)

In [24]:
products = products.iloc[:,[0,2,3,1,4,5]]

In [25]:
display(orders.head(),products.head(),order_products.head())

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,1,2,8,0
1,2398795,1,2,3,7,15
2,473747,1,3,3,12,21
3,2254736,1,4,4,7,29
4,431534,1,5,4,15,28


,product_id,aisle_id,department_id,product_name,aisle,department
0,1,61,19,Chocolate Sandwich Cookies,cookies cakes,snacks
1,2,104,13,All-Seasons Salt,spices seasonings,pantry
2,3,94,7,Robust Golden Unsweetened Oolong Tea,tea,beverages
3,4,38,1,Smart Ones Classic Favorites Mini Rigatoni Wit...,frozen meals,frozen
4,5,5,13,Green Chile Anytime Sauce,marinades meat preparation,pantry


,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


## New table
- `orders`
- `order_products` 
- `products` `merge()` with `aisles` and `departments`

### c. Basic Descriptive Statistics:
   - Calculate summary statistics for numerical features.
   - Examine the distribution of categorical features.

In [26]:
summary_statistics = pd.concat([order_products.iloc[:,[2,3]].describe(percentiles=[0.9,0.99]) , orders.iloc[:,2:6].describe(percentiles=[0.9,0.99])],axis=1).astype(int)
summary_statistics

,add_to_cart_order,reordered,order_number,order_dow,order_hour_of_day,days_since_prior_order
count,33819106,33819106,3421083,3421083,3421083,3421083
mean,8,0,17,2,13,10
std,7,0,17,2,4,9
min,1,0,1,0,0,0
50%,6,1,11,3,13,7
90%,18,1,42,6,19,30
99%,33,1,83,6,23,30
max,145,1,100,6,23,30


In [27]:
categories = products[["department","aisle","product_name"]].sort_values(["department","aisle","product_name"])

In [28]:
categories.groupby(["department","aisle"]).count().join(
    categories.groupby(["department"]).nunique().rename(columns={
    "aisle": "aisle_in_dep",
    "product_name": "products_in_dep"
    })
).rename(columns={
    "product_name": "products_in_aisle",
}).reset_index().set_index(["department","aisle_in_dep", "products_in_dep","aisle"])

products_in_aisle
department aisle_in_dep products_in_dep aisle                                        
alcohol    5            1054            beers coolers                             385
                                        red wines                                 232
                                        specialty wines champagnes                 95
                                        spirits                                   195
                                        white wines                               147
...                                                                               ...
snacks     11           6264            ice cream toppings                         85
                                        mint gum                                  168
                                        nuts seeds dried fruit                    582
                                        popcorn jerky                             316
                                        trail mix snack mix                        69

[134 rows x 1 columns]

In [29]:
pd.set_option('display.max_row', False)

## 2. Exploratory Data Analysis (EDA)
### a. Customer Behavior:
 - Average number of orders per user.
 - Average time between orders for each user.
 - Number of orders placed by each customer.
 - Customer segments based on purchase frequency.

In [30]:
orders["order_number"].count() / orders["user_id"].nunique()


16.590367054784224

In [31]:
orders.groupby("user_id").mean().astype(int)

,order_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
user_id,,,,,
1,1923450,6,2,10,17
2,1768104,8,2,10,15
3,1934104,7,1,16,11
4,1507834,3,4,12,14
5,1849578,3,1,15,9
6,1227931,2,3,17,10
7,1782058,11,1,13,9
8,1757721,2,3,5,17
9,1491936,2,3,13,16


In [32]:
orders.groupby("user_id")["order_number"].count()

user_id
1         11
2         15
3         13
4          6
5          5
6          4
7         21
8          4
9          4
10         6
11         8
12         6
          ..
206198     8
206199    20
206200    24
206201    33
206202    23
206203     6
206204     5
206205     4
206206    68
206207    17
206208    50
206209    14
Name: order_number, Length: 206209, dtype: int64

In [33]:
freq = orders.groupby("user_id")["days_since_prior_order"].mean().astype(int)
bins = np.arange(freq.min()-1 , freq.max() + 2,4)
segments = pd.cut(freq,bins).reset_index().rename(columns={"days_since_prior_order":"segments"}).sort_values("segments").set_index("segments")
segments

,user_id
segments,
"(-1, 3]",146737
"(-1, 3]",97805
"(-1, 3]",132783
"(-1, 3]",119610
"(-1, 3]",86559
"(-1, 3]",204022
"(-1, 3]",21969
"(-1, 3]",183016
"(-1, 3]",163035


In [34]:
segments.groupby("segments",observed=True).count()

,user_id
segments,
"(-1, 3]",5645
"(3, 7]",34086
"(7, 11]",46637
"(11, 15]",48216
"(15, 19]",42179
"(19, 23]",25162
"(23, 27]",4284


### b. Product Analysis:
 - Identify most popular products by frequency.
 - Determine average order size (number of items per order).

In [35]:
product_frequency = order_products.product_id.value_counts().reset_index()

In [37]:
product_frequency.merge(products).iloc[:,[4,1]]

,product_name,count
0,Banana,491291
1,Bag of Organic Bananas,394930
2,Organic Strawberries,275577
3,Organic Baby Spinach,251705
4,Organic Hass Avocado,220877
5,Organic Avocado,184224
6,Large Lemon,160792
7,Strawberries,149445
8,Limes,146660
...,...,...


In [38]:
int(order_products.shape[0] / order_products.order_id.nunique())

10

### c. Temporal Patterns:

 - Analyze orders by day of the week and hour of the day.
 - Explore seasonal trends or patterns in purchasing behavior.
 - Months with higher order volumes.


In [39]:
days=["Sunday","Monday"," Tuesday", "Wednesday" ,"Thursday", "Friday", "Saturday"]

orders["weekday_name"] = [days[x] for x in orders.order_dow]

In [40]:
def generate_date(start_date:pd.Timestamp,order_dow,order_hour_of_day):
    date_list = [start_date.replace(hour=order_hour_of_day.iloc[0])]
    last_date = start_date
   
    for index in range(1,len(order_dow)):
        
        first_day = order_dow.iloc[index - 1]
        second_day = order_dow.iloc[index]
        timedelta = 0
        
        if second_day < first_day:
            timedelta = second_day + 7 - first_day
        else:
            timedelta = second_day - first_day
        timedelta = timedelta % 7
        last_date = last_date + pd.Timedelta(days = timedelta)
        date_list.append(last_date.replace(hour=order_hour_of_day.iloc[index]))
    return date_list

In [41]:
sorted_orders = orders.sort_values("order_dow")

In [42]:
pd.set_option("display.max_columns",None)
pd.set_option("display.max_row",10)

sort = orders.sort_values("order_id")
sort = sort.iloc[:100,:]
sort["date"] = generate_date(pd.Timestamp(year=2024, month=1, day=4, hour=5),sort.order_dow[:100],sort.order_hour_of_day[:100])
orders.iloc[0:20]

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,weekday_name
0,2539329,1,1,2,8,0,Tuesday
1,2398795,1,2,3,7,15,Wednesday
2,473747,1,3,3,12,21,Wednesday
3,2254736,1,4,4,7,29,Thursday
4,431534,1,5,4,15,28,Thursday
...,...,...,...,...,...,...,...
15,1673511,2,5,3,11,8,Wednesday
16,1199898,2,6,2,9,13,Tuesday
17,3194192,2,7,2,12,14,Tuesday
18,788338,2,8,1,15,27,Monday


In [43]:
pivot = pd.pivot_table(
    data = sorted_orders,
    values="order_id",
    index="weekday_name",
    columns="order_hour_of_day",
    aggfunc="count",
    margins=True,
    margins_name='Grand Total',
    sort=False)
pivot.columns = pivot.columns[0:-1].sort_values().append(pd.Index([pivot.columns[-1]])) 
pivot

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,Grand Total
weekday_name,,,,,,,,,,,,,,,,,,,,,,,,,
Sunday,11246,53954,49463,14423,51443,6887,40798,54552,53849,51035,39753,28108,18277,12410,22654,48465,29572,3329,3936,1409,813,2398,1168,963,600905
Monday,8992,46403,44761,11943,47079,5620,51908,46764,46728,51584,36792,34116,16281,16571,22145,55671,28977,5370,3674,1105,809,1830,1607,748,587478
Tuesday,8146,37469,37541,10653,35780,5358,36314,37173,36650,38128,32151,24635,15039,13245,20084,39230,26470,4758,3059,943,744,1572,1399,719,467260
Wednesday,8242,35990,35273,10278,33455,5181,32312,34773,34161,35215,30368,22553,13795,12396,19249,36040,25001,4562,2952,953,719,1495,1355,654,436972
Thursday,8812,34222,34093,10796,32249,5645,31409,33625,32751,33857,29378,21814,14186,12493,19350,35034,24425,4401,2642,899,730,1512,1330,686,426339
Friday,7498,37508,35860,9515,35714,5265,34232,37407,36296,37915,29955,24015,13322,13434,18741,38313,24310,4866,3189,1016,910,1672,1574,841,453368
Saturday,8532,38093,35562,10501,37121,6087,30839,38748,37564,36994,30398,22960,13392,11319,18346,35665,24157,3243,3306,1214,802,1919,1136,863,448761
Grand Total,61468,283639,272553,78109,272841,40043,257812,283042,277999,284728,228795,178201,104292,91868,140569,288418,182912,30529,22758,7539,5527,12398,9569,5474,3421083



### d. Basket Analysis:

 - Identify most frequently co-purchased items.
 - Products often bought together on weekends vs. weekdays.

In [44]:
freq_table = order_products[["order_id","product_id"]]

In [45]:
pd.set_option("display.max_row",10) 


In [46]:
def print_num(num):
    print("{:,}".format(num))

print_num(order_products.shape[0])
print_num(orders.shape[0])
print_num(order_products.groupby("order_id").count().shape[0])

33,819,106
3,421,083
3,346,083


In [47]:
import itertools as s
f = ["Banana","Bag of Organic Bananas","Organic Strawberries","Organic Baby Spinach","Organic Hass Avocado"]
for i in s.combinations(f, 2):
    print(i)

('Banana', 'Bag of Organic Bananas')
('Banana', 'Organic Strawberries')
('Banana', 'Organic Baby Spinach')
('Banana', 'Organic Hass Avocado')
('Bag of Organic Bananas', 'Organic Strawberries')
('Bag of Organic Bananas', 'Organic Baby Spinach')
('Bag of Organic Bananas', 'Organic Hass Avocado')
('Organic Strawberries', 'Organic Baby Spinach')
('Organic Strawberries', 'Organic Hass Avocado')
('Organic Baby Spinach', 'Organic Hass Avocado')


## 3. Business Questions and Analysis
### a. Popular Products:

 - Analyze sales distribution of top-selling products.
 - Identify top 5 products commonly added to the cart first.
 - Top 10 product pairs frequently purchased together.


In [48]:
top_selling = order_products["product_id"]\
                .value_counts().to_frame() \
                .merge(right=products,on="product_id")
top_selling[["product_name","count"]]

,product_name,count
0,Banana,491291
1,Bag of Organic Bananas,394930
2,Organic Strawberries,275577
3,Organic Baby Spinach,251705
4,Organic Hass Avocado,220877
...,...,...
49680,Coconut Bliss Pineapple Coconut,1
49681,"Salsa, Black Bean",1
49682,Sweetart Jelly Beans,1
49683,The Ultimate Caramel Suace,1


In [49]:
product_ids = order_products\
            .query("add_to_cart_order == 1")\
            .groupby("product_id").count()\
            .nlargest(5,"add_to_cart_order")
            
top_five = product_ids.merge(right=products,on="product_id").drop(["reordered","aisle_id","department_id","order_id"],axis=1)
top_five = top_five.iloc[:,[0,2,1,3,4]].rename({"add_to_cart_order":"num_of_item_bought_first"},axis=1)
top_five

,product_id,product_name,num_of_item_bought_first,aisle,department
0,24852,Banana,115521,fresh fruits,produce
1,13176,Bag of Organic Bananas,82877,fresh fruits,produce
2,27845,Organic Whole Milk,32071,milk,dairy eggs
3,21137,Organic Strawberries,28875,fresh fruits,produce
4,47209,Organic Hass Avocado,24913,fresh fruits,produce


### b. Reorder Behavior:

 - Analyze the reordered column to understand repeat purchase behavior.
 - Products reordered the most.
 - Reorder behavior based on day of the week and days since prior order.
 - How the number of items in the cart impacts the likelihood of reordering.

In [51]:
order_products.query("add_to_cart_order == 2 & reordered == 1")

,order_id,product_id,add_to_cart_order,reordered
1,1,11109,2,1
9,36,19660,2,1
33,98,19731,2,1
82,112,21174,2,1
93,170,37766,2,1
...,...,...,...,...
33819044,3421075,18721,2,1
33819052,3421076,25718,2,1
33819064,3421078,24852,2,1
33819074,3421080,4932,2,1


In [52]:
megred = order_products.merge(orders).drop(orders.columns[1:])

KeyError: "['user_id', 'order_number', 'order_dow', 'order_hour_of_day', 'days_since_prior_order', 'weekday_name'] not found in axis"

In [ ]:
megred

### c. Customer Segmentation:
 - Segment customers based on the total amount they’ve spent on orders.
 - Identify customers who haven’t placed an order in the last 30 days.
 - Percentage of customers who have churned in the past quarter.

In [ ]:
products.query("aisle.str.contains('chu')"

### d. Department and Aisle Analysis:

 - Best-selling department and aisle breakdown.
 - What is the “produce” department? Break it down by aisle.
 - Differences in purchasing behavior based on different departments or aisles
 

In [58]:
department_and_aisle = top_selling[["department","aisle","count"]]

In [59]:
dep_count = department_and_aisle[["department","count"]].groupby("department").sum().rename({"count":"dep_count"},axis=1)
aisle_count = department_and_aisle.groupby(["department","aisle"]).sum().rename({"count":"aisle_count"},axis=1)

In [60]:
best_selling = dep_count.join(aisle_count).reset_index()

In [61]:
best_selling = best_selling.set_index(["department","dep_count","aisle"]).sort_index(level = 1,ascending = False)

In [68]:
pd.set_option("display.max_row",None)
best_selling

aisle_count
department      dep_count aisle                                     
produce         9888378   packaged vegetables fruits         1843806
                          packaged produce                    289488
                          fresh vegetables                   3568630
                          fresh herbs                         393793
                          fresh fruits                       3792661
dairy eggs      5631067   yogurt                             1507583
                          specialty cheeses                    86364
                          soy lactosefree                     664493
                          refrigerated pudding desserts        39420
                          packaged cheese                    1021462
                          other creams cheeses                320827
                          milk                                923659
                          eggs                                472009
                          cream                               330358
                          butter                              264892
snacks          3006412   trail mix snack mix                  37053
                          popcorn jerky                       170441
                          nuts seeds dried fruit              319019
                          mint gum                             23626
                          ice cream toppings                   11706
                          fruit vegetable snacks              182498
                          energy granola bars                 473835
                          crackers                            478430
                          cookies cakes                       244045
                          chips pretzels                      753739
                          candy chocolate                     312020
beverages       2804175   water seltzer sparkling water       878150
                          tea                                 258717
                          soft drinks                         373816
                          refrigerated                        599109
                          juice nectars                       348283
                          energy sports drinks                108357
                          coffee                              215467
                          cocoa drink mixes                    22276
frozen          2336858   ice cream ice                       521101
                          frozen vegan vegetarian             104096
                          frozen produce                      545107
                          frozen pizza                        169879
                          frozen meat seafood                  75083
                          frozen meals                        408520
                          frozen juice                          5147
                          frozen dessert                       20241
                          frozen breakfast                    242492
                          frozen breads doughs                 41740
                          frozen appetizers sides             203452
pantry          1956819   spreads                             301502
                          spices seasonings                   221371
                          salad dressing toppings             104050
                          preserved dips spreads              113879
                          pickled goods olives                117521
                          oils vinegars                       256086
                          marinades meat preparation           65415
                          honeys syrups nectars                69790
                          doughs gelatins bake mixes          110847
                          condiments                          231792
                          baking supplies decor                24786
                          baking ingredients           